In [24]:
from ipymarkup import show_markup
from dataloader import DataLoader
from random import sample
from IPython.display import display
from ipymarkup import show_markup
from yargy import rule, and_, or_
from yargy.interpretation import fact, attribute
from yargy.predicates import eq, gte, lte, length_eq, dictionary, normalized
from IPython.display import display

RangDateFact = fact('RangDateFact', ['startyear', 'startmonth', 'startday','endyear', 'endmonth', 'endday',
                         attribute('current_era', True)])

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12,
}

START_MONTH_NAME = dictionary(MONTHS).interpretation(RangDateFact.startmonth.normalized().custom(MONTHS.__getitem__))
END_MONTH_NAME = dictionary(MONTHS).interpretation(RangDateFact.endmonth.normalized().custom(MONTHS.__getitem__))

START_MONTH = and_(gte(1), lte(12)).interpretation(RangDateFact.startmonth.custom(int))
END_MONTH = and_(gte(1), lte(12)).interpretation(RangDateFact.endmonth.custom(int))

START_DAY = and_(gte(1),lte(31)).interpretation(RangDateFact.startday.custom(int))
END_DAY = and_(gte(1),lte(31)).interpretation(RangDateFact.endday.custom(int))

START_YEAR = and_(gte(2000),lte(2100)).interpretation(RangDateFact.startyear.custom(int))
END_YEAR = and_(gte(2000),lte(2100)).interpretation(RangDateFact.endyear.custom(int))

START_YEAR_SHORT = and_(gte(1),lte(99)).interpretation(RangDateFact.startyear.custom(int))
END_YEAR_SHORT = and_(gte(1),lte(99)).interpretation(RangDateFact.endyear.custom(int))

START_PREFIX = or_(rule(normalized('c')))
END_PREFIX = or_(rule(normalized('по')),  rule(normalized('на')), rule(normalized('и')), rule(normalized('до')), rule(eq('-')))


RANGE_DATE_PARSER = or_(
    #5.09 / 5.09.2019
    rule(
        START_DAY,
        '.',
        START_MONTH,
        '.',
        or_(
            START_YEAR,
            START_YEAR_SHORT
        ).optional()
    ),
    #8.09-8.09 / 8.09.2019-9.09.2019
    rule(
        START_DAY,
        '.',
        START_MONTH,
        eq('.').optional(),
        or_(
            START_YEAR,
            START_YEAR_SHORT
        ).optional(),
        END_PREFIX,
        END_DAY,
        '.',
        END_MONTH,
        eq('.').optional(),
        or_(
            END_YEAR,
            END_YEAR_SHORT
        ).optional()
    ),
    #8.09 по 8.09 / 8.09.2019 по 8.09.2019
    rule(
        START_PREFIX.optional(),
        START_DAY,
        '.',
        START_MONTH,
        eq('.').optional(),
        or_(
            START_YEAR,
            START_YEAR_SHORT
        ).optional(),
        END_PREFIX,
        END_DAY,
        '.',
        END_MONTH,
        eq('.').optional(),
        or_(
            END_YEAR,
            END_YEAR_SHORT
        ).optional()
    ),
    #5 стентября
    rule(
        START_DAY,
        START_MONTH_NAME,
        or_(
            START_YEAR,
            START_YEAR_SHORT
        ).optional()
    ),
    # 5 по 8 сентября /  5 сентября по 8 сентября
    rule(
        START_PREFIX.optional(),
        START_DAY,
        START_MONTH_NAME.optional(),
        or_(
            START_YEAR,
            START_YEAR_SHORT
        ).optional(),
        END_PREFIX,
        END_DAY,
        END_MONTH_NAME,
        or_(
            END_YEAR,
            END_YEAR_SHORT
        ).optional()
    )
).interpretation(
    RangDateFact
)


lines = DataLoader().getTextList()
parser = Parser(RANGE_DATE_PARSER)

for line in lines[:80]:
    result_list = []
    line = line.lower()
    matches = list(parser.findall(line))
    print('----------------------------------------------------------------------')
    for match in matches:
        if match is not None:
            try:
                #result_list.append(match.fact.gender)
#                 print("startyear: " + match.fact.startyear)
#                 print("startmonth: " + match.fact.startmonth)
#                 print("startday: " + match.fact.startday)
                 print(" ")
#                 print("endyear: " + match.fact.endyear)
#                 print("endmonth: " + match.fact.endmonth)
#                 print("endday: " + match.fact.endday)
            except KeyError:
                print("Gender: KeyError")             
    show_markup(line, [_.span for _ in matches])
    result_list = list(dict.fromkeys(result_list))
    print(result_list)


----------------------------------------------------------------------
 


BoxMarkup('москва  5 октября  для ролика-презинтации (внутренняя разовая трансляция)  требуется актриса:  девушка - 18-25л, светлые глаза, светлые волосы, максимальное сходство лица с референсом (прилагается) - оплата 3000р за смену  внимание: заявки отправлять на вацап вот на этот номер!!!  89276358015 татьяна  либо на почту: t89276358015@gmail.com  тема: девушка  утверждаем по видеовизитке  в заявке: фио, видеовизитку, актуальные фото, возраст, рост, размер одежды, обувь',
          [Span(8, 17, None)])

[]
----------------------------------------------------------------------


BoxMarkup(' приглашаю девушек-моделей 13-20 лет , стройных, спортивного телосложения , рост значения не имеет, для работы в документальном проекте «искусство массажа» , съемка в комфортной студии в москве ,оплата в день съемки - 20000 руб. в подарок - профессиональное портфолио. отличное предложение для вашего "старта". просьба высылать фото в полный рост на\xa0   e-mail:\xa0  t-f-p@mail.ru  ',
          [])

[]
----------------------------------------------------------------------


...

[]
----------------------------------------------------------------------
 


BoxMarkup("для съёмки видеоролика требуются игровые дети:      дата съёмки - 30 сентября  примерная занятость - с 14:00 до 22:00 (точное время будет чуть позже)   территориально метро достоевская.         мальчик брюнет 10-14 лет  гонорар 5000  форма одежды: серая футболка, джинсы, кеды.   девочка брюнетка 10-14 лет  гонорар 5000  форма одежды: футболка светло-розовая, юбка  до колена/выше колена, кроссовки белые.  заявки отправлять строго на почту s.shukshina90@mail.ru  в теме письма обязательно указать роль 'ребенок'.   в заявке указать:   1. фио ребенка/фио родителя  2. возраст, дата рождения ребенка  3. рост, размер одежды/обуви ребенка  4. опыт съёмок(если есть)   5. контактный номер телефона родителя  6. ссылки на соц сети ребенка(если есть)   7. обязательно 5-10   фото ребенка, где его хорошо видно без тонны фотошопа.   8. видеовизитка, если есть.      просьба ответственно относиться к подаче заявки!",
          [Span(66, 77, None)])

[]
----------------------------------------------------------------------
 


BoxMarkup('москва заявки на почту 🚶\u200d♀   т/с ип пирогова 2 сезон   только те, кто не снимались в ип пирогова!      дата съемки: 01 октября      требуется: актриса или хорошая игровая девушка   типаж: симпатичная, милая   игровой возраст: 20 - 25 лет   по сюжету: девушка - 7 ( заплаканная )      только рф/оплата 10 000 руб - 20% агентские      утверждает режиссер по видео - пробе   другого способа не предполагается      в пробе играете текст девушки - 7   по сюжету: главная героиня успокаивает девушку - 7,   из - за того, что девушка - 7 расстроилась - перед свадьбой все идет не так.      👉🏻текст для видео - пробы      героиня сидит за столиком. напротив неё – заплаканная девушка-7   героиня успокаивает девушка-7      героиня (участливо, с душой)   - не переживайте… у вас прекрасное платье, хороший ресторан!   будет отличная свадьба!      девушка-7   - правда?      героиня кивает – «конечно», девушка-7 успокаивается      девушка-7   спасибо… вы лучше, чем мой психолог…      видео - пробу отправлять на почту: pirogova@artcasta.ru   в теме письма указываете: девушка - 7      1. видео - проба   2. ф.и.о / ваш номер телефона   3. возраст/рост/размер одежды/обуви   4. 3-4 фото      спасибо 😊   ксения',
          [Span(116, 126, None)])

[]
